In [5]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pyautogui as p
import time

import requests



### Historical data from the odds api, but i have to pay. Worst case ill pay, if im not able to scrap from betting pros

In [4]:
import requests
base_url = "https://api.the-odds-api.com/v4/sports/basketball_nba/events/"
API_KEY = '8aa8bb1fc5d56c98b45d6f3f58beab70'
Sport = 'basketball_nba'
Regions = 'us'
Market = 'h2h,odds'
Odds_format = 'american'
Date_format = 'iso' 
date = '2021-10-22T22:45:00Z'
id = '34e829d65b6a0cbb49573338a86772ec'

url = (
    f"https://api.the-odds-api.com/v4/historical/sports/{Sport}/events"
    f"?apiKey={API_KEY}"
    f"&date={date}"
    f"&regions={Regions}"
)

response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to get events: {response.status_code}, {response.text}")
else:
    events = response.json()
    for event in events:
        print(event['id'], event['commence_time'], event['home_team'], event['away_team'])

Failed to get events: 401, {"message":"Historical odds are only available on paid usage plans. See usage plans at https://the-odds-api.com","error_code":"HISTORICAL_UNAVAILABLE_ON_FREE_USAGE_PLAN","details_url":"https://the-odds-api.com/liveapi/guides/v4/api-error-codes.html#historical-unavailable-on-free-usage-plan"}



### Grabs players odds for the day(US all boookmakers, DFS is prizepicks and underdogs)

In [1]:
from NBAPropFinder.NBAPropFinder import NBAPropFinder

nba_props = NBAPropFinder()

Scraping Odds API...
Organizing Data...


In [2]:
prizePicks = nba_props.dataframe
prizePicks
# df.to_csv('./PROPS_DATA/Playoffs_PP(5_13_25)')

,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,DraftKings,player_points,Shai Gilgeous-Alexander,Over,34.5,-110
1,DraftKings,player_points,Shai Gilgeous-Alexander,Under,34.5,-120
2,DraftKings,player_points,Jalen Williams,Over,21.5,-105
3,DraftKings,player_points,Jalen Williams,Under,21.5,-125
4,DraftKings,player_points,Pascal Siakam,Over,19.5,-110
...,...,...,...,...,...,...
1505,DraftKings,player_blocks_steals,Andrew Nembhard,Under,1.5,-135
1506,DraftKings,player_blocks_steals,Aaron Nesmith,Over,1.5,114
1507,DraftKings,player_blocks_steals,Aaron Nesmith,Under,1.5,-145
1508,DraftKings,player_blocks_steals,Isaiah Hartenstein,Over,1.5,154


In [3]:
prizePicks.to_csv('./PROPS_DATA/Playoffs_US(6_9_25).csv')

In [7]:
prizePicks = pd.read_csv('./PROPS_DATA/Playoffs_US(6_9_25).csv')
prizePicks.drop(columns=['Unnamed: 0'], inplace=True)
prizePicks

,BOOKMAKER,CATEGORY,NAME,OVER/UNDER,LINE,ODDS
0,DraftKings,player_points,Shai Gilgeous-Alexander,Over,34.5,-110
1,DraftKings,player_points,Shai Gilgeous-Alexander,Under,34.5,-120
2,DraftKings,player_points,Jalen Williams,Over,21.5,-105
3,DraftKings,player_points,Jalen Williams,Under,21.5,-125
4,DraftKings,player_points,Pascal Siakam,Over,19.5,-110
...,...,...,...,...,...,...
1505,DraftKings,player_blocks_steals,Andrew Nembhard,Under,1.5,-135
1506,DraftKings,player_blocks_steals,Aaron Nesmith,Over,1.5,114
1507,DraftKings,player_blocks_steals,Aaron Nesmith,Under,1.5,-145
1508,DraftKings,player_blocks_steals,Isaiah Hartenstein,Over,1.5,154


### Grab player data using fetchplayerstats class

In [ ]:
from NBAData.fetchPlayersStats import FetchPlayersStats

nba = FetchPlayersStats()
data = nba.getCompleteStats(season='2022-23', season_type='Playoffs', sleep_time=1, max_workers=4)
data

In [145]:
# data.to_csv('PLAYOFF_DATA/PLAYOFFS_25.csv')

In [143]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_24.csv')
data = data.drop(columns=['Unnamed: 0'])
data['OPP_TEAM_ID'] = data['OPP_TEAM_ID'].fillna(0).astype(int)
data.head()

,PLAYER_NAME,PLAYER_ID,MATCHUP,TEAM_ABBREVIATION,TEAM_ID,OPP_ABBREVIATION,HOME_GAME,GAME_ID,GAME_DATE,WL,MIN,PTS,AST,REB,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,STL,BLK,TOV,PF,PLUS_MINUS,FANTASY_PTS,PointsPerShot,eFG,START_POSITION,OFF_RATING,DEF_RATING,NET_RATING,OREB_PCT,DREB_PCT,REB_PCT,AST_PCT,AST_TOV,USG_PCT,TS_PCT,E_PACE,PACE,PIE,PACE_PER40,TEAM_GAME_DATE,TEAM_MIN,TEAM_FGM,TEAM_FGA,TEAM_FG_PCT,TEAM_FG3M,TEAM_FG3A,TEAM_FG3_PCT,TEAM_FTM,TEAM_FTA,TEAM_FT_PCT,TEAM_OREB,TEAM_DREB,TEAM_REB,TEAM_AST,TEAM_STL,TEAM_BLK,TEAM_TOV,TEAM_PF,TEAM_PTS,OPP_DEF_RATING,OPP_STL,OPP_BLK,OPP_REB,OPP_FG_PCT,OPP_TEAM_ID,TEAM_OFF_RATING,TEAM_PACE,GAME_PACE,OPP_PACE
0,Darius Garland,1629636,CLE vs. ORL,CLE,1610612739,ORL,1,42300131,2024-04-20,W,32,14,8,3,6,11,0.545,2,6,0.333,0,0,NaN,0,3,1,0,5,4,8,27.6,1.273,1.273,G,100.0,87.9,12.1,0.000,0.067,0.042,0.400,1.6,0.229,0.636,101.08,97.82,0.145,81.52,"APR 20, 2024",240,36,81,0.444,8,30,0.267,17,21,0.810,10,44,54,21,9,4,17,21,97,99.753188,12.0,7.0,40.0,0.325581,1610612753,99.753188,97.24,98.72,100.20
1,Kelly Oubre Jr.,1626162,PHI @ NYK,PHI,1610612755,NYK,0,42300111,2024-04-20,L,39,10,1,3,3,7,0.429,1,2,0.500,3,4,0.750,1,2,5,1,2,3,6,31.1,1.142,1.142,F,132.4,120.5,11.8,0.029,0.047,0.039,0.034,0.5,0.139,0.571,91.36,89.20,0.058,74.34,"APR 20, 2024",240,36,81,0.444,12,35,0.343,20,22,0.909,9,24,33,17,8,7,11,21,104,112.214070,6.0,7.0,55.0,0.395604,1610612752,112.214070,92.68,92.00,91.32
2,Joel Embiid,203954,PHI @ NYK,PHI,1610612755,NYK,0,42300111,2024-04-20,L,37,29,6,8,8,22,0.364,2,8,0.250,11,12,0.917,3,5,2,0,2,3,14,51.6,1.063,1.063,C,143.3,120.6,22.7,0.103,0.125,0.116,0.250,3.0,0.387,0.532,90.96,88.67,0.154,73.89,"APR 20, 2024",240,36,81,0.444,12,35,0.343,20,22,0.909,9,24,33,17,8,7,11,21,104,112.214070,6.0,7.0,55.0,0.395604,1610612752,112.214070,92.68,92.00,91.32
3,Kyle Lowry,200768,PHI @ NYK,PHI,1610612755,NYK,0,42300111,2024-04-20,L,39,18,2,4,5,9,0.556,4,7,0.571,4,4,1.000,2,2,1,1,1,4,6,30.8,1.673,1.673,G,132.4,120.5,11.8,0.059,0.047,0.052,0.074,2.0,0.139,0.836,91.36,89.20,0.099,74.34,"APR 20, 2024",240,36,81,0.444,12,35,0.343,20,22,0.909,9,24,33,17,8,7,11,21,104,112.214070,6.0,7.0,55.0,0.395604,1610612752,112.214070,92.68,92.00,91.32
4,Wendell Moore Jr.,1631111,MIN vs. PHX,MIN,1610612750,PHX,1,42300161,2024-04-20,W,3,0,0,0,0,0,NaN,0,0,NaN,0,0,NaN,0,0,0,0,0,0,3,0.0,0.000,0.000,NaN,116.7,66.7,50.0,0.000,0.000,0.000,0.000,0.0,0.000,0.000,100.25,109.37,0.000,91.14,"APR 20, 2024",240,44,88,0.500,12,32,0.375,20,22,0.909,13,39,52,26,8,2,14,24,120,121.605188,9.0,6.0,28.0,0.440000,1610612756,121.605188,98.68,98.34,98.00


In [ ]:
from NBAData.features import add_PTS_features
from NBAData.TBD import *

data['STARTING'] = data['START_POSITION'].apply(lambda x: 1 if x in ['G','F','C'] else 0)
data = add_PTS_features(data)
data = assign_position(data)
data

In [142]:
data.to_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')

### If you already have the data just run the cell below

In [100]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
prizePicks = pd.read_csv('./PROPS_DATA/Playoffs_US(6_9_25).csv')
Props = prizePicks[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[prizePicks['CATEGORY'] == 'player_points']

### For Single Bets

In [103]:
import numpy as np
import pandas as pd
import scipy.stats as stats

results = []

for idx, row in Props.iterrows():
    name = row['NAME']
    bookmaker = row['BOOKMAKER']
    category = row['CATEGORY']
    line = row['LINE']
    over_under = row['OVER/UNDER']
    odds = row['ODDS']

    # Filter player data
    df = data.loc[data['PLAYER_NAME'] == name]
    if df.empty:
        continue

    # Mean and std dev of actual results
    mean = round(df['PTS'].mean(), 2)
    std = round(df['PTS'].std(), 2)
    if std == 0:
        continue

    # Z-score-based probability
    z_score = (line - mean) / std
    norm_prob = 1 - stats.norm.cdf(z_score)
    if over_under == "Under":
        norm_prob = 1 - norm_prob

    # Monte Carlo simulation
    sim_points = np.random.normal(mean, std, 10000)
    sim_prob = np.mean(sim_points > line)
    if over_under == "Under":
        sim_prob = 1 - sim_prob

    # Weighted average of sim and norm prob
    final_prob = 0.8 * sim_prob + 0.2 * norm_prob
    
    # EV calculation
    stake = 100
    profit = (odds / 100) * stake if odds > 0 else (100 / abs(odds)) * stake
    payout = stake + profit
    ev = (final_prob * profit) - ((1 - final_prob) * stake)

    # Kelly Criterion
    def kelly_criterion(prob, payout, stake):
        net_profit = payout - stake
        loss_prob = 1 - prob
        kelly = (net_profit * prob - loss_prob * stake) / net_profit
        return max(0, round(kelly, 4))

    kelly = kelly_criterion(final_prob, payout, stake)

    results.append({
        'NAME': name,
        'BOOKMAKER': bookmaker,
        'CATEGORY': category,
        'LINE': line,
        'OVER/UNDER': over_under,
        'ODDS': odds,
        'SIM PROB': round(final_prob, 3),
        'EV': round(ev, 2),
        'KELLY CRITERION': kelly,
    })

current = pd.DataFrame(results).sort_values(by='EV', ascending=False).reset_index(drop=True)
current
# current.to_csv('PROPS_EV/singleBets(6_9_25).csv')

,NAME,BOOKMAKER,CATEGORY,LINE,OVER/UNDER,ODDS,SIM PROB,EV,KELLY CRITERION
0,Shai Gilgeous-Alexander,Bovada,player_points,29.5,Under,210,0.467,44.80,0.2134
1,Isaiah Hartenstein,BetRivers,player_points,7.5,Over,112,0.683,44.75,0.3995
2,Shai Gilgeous-Alexander,Bovada,player_points,31.5,Under,150,0.569,42.21,0.2814
3,Isaiah Hartenstein,BetRivers,player_points,6.5,Over,-121,0.778,42.11,0.5095
4,Shai Gilgeous-Alexander,Bovada,player_points,30.5,Under,175,0.506,39.18,0.2239
...,...,...,...,...,...,...,...,...,...
373,Isaiah Hartenstein,FanDuel,player_points,6.5,Under,102,0.221,-55.38,0.0000
374,Isaiah Hartenstein,BetRivers,player_points,6.5,Under,-109,0.225,-56.93,0.0000
375,Shai Gilgeous-Alexander,Bovada,player_points,38.5,Over,170,0.152,-58.91,0.0000
376,Isaiah Hartenstein,Bovada,player_points,5.5,Under,170,0.151,-59.28,0.0000


## *PrizePicks & Underdog*
## - 2 leg EVs since we cant do single bets on those platforms

In [105]:
data = pd.read_csv('PLAYOFF_DATA/PLAYOFFS_25_FEATURES.csv')
prizePicks = pd.read_csv('./PROPS_DATA/Playoffs_PP_DFS(6_9_25).csv')
pointsProps = prizePicks[['NAME', 'BOOKMAKER', 'CATEGORY','LINE','OVER/UNDER', 'ODDS']].loc[prizePicks['CATEGORY'] == 'player_points']

In [106]:
from scipy.stats import norm
# This is the monte carlo simulation for the pairs, then multiplies with the z-score probability, followed by the kelly criterion

def zscore_prob(mean, std, line, side="over"):
    z = (line - mean) / std
    prob = 1 - norm.cdf(z)
    return prob if side == "over" else 1 - prob

def monte_carlo_sim(name1, name2, simulations=10000, stake=100, payout=300, weight=0.8):
    df1 = data.loc[data['PLAYER_NAME'] == name1]
    df2 = data.loc[data['PLAYER_NAME'] == name2]

    line1 = pointsProps.loc[pointsProps['NAME'] == name1, 'LINE'].values[0]
    line2 = pointsProps.loc[pointsProps['NAME'] == name2, 'LINE'].values[0]

    mean1, std1 = df1['PTS'].mean(), df1['PTS'].std()
    mean2, std2 = df2['PTS'].mean(), df2['PTS'].std()
    
    if std1 == 0 or std2 == 0:
        return None

    # Monte Carlo simulation
    sim1 = np.random.normal(mean1, std1, simulations)
    sim2 = np.random.normal(mean2, std2, simulations)

    over1, under1 = sim1 > line1, sim1 <= line1
    over2, under2 = sim2 > line2, sim2 <= line2

    mc_probs = {
        'OVER/OVER': np.mean(over1 & over2),
        'UNDER/UNDER': np.mean(under1 & under2),
        'OVER/UNDER': np.mean(over1 & under2),
        'UNDER/OVER': np.mean(under1 & over2),
    }

    # Z-score-based probabilities
    z_over1 = zscore_prob(mean1, std1, line1, side='over')
    z_under1 = 1 - z_over1
    z_over2 = zscore_prob(mean2, std2, line2, side='over')
    z_under2 = 1 - z_over2

    z_probs = {
        'OVER/OVER': z_over1 * z_over2,
        'UNDER/UNDER': z_under1 * z_under2,
        'OVER/UNDER': z_over1 * z_under2,
        'UNDER/OVER': z_under1 * z_over2,
    }

    # Blended probabilities
    probs = {
        k: weight * mc_probs[k] + (1 - weight) * z_probs[k]
        for k in mc_probs
    }

    evs = {k: round((probs[k] * payout) - stake, 2) for k in probs}

    return {
        'PLAYER 1': name1,
        'PLAYER 2': name2,
        'OVER/OVER': {'prob': round(probs['OVER/OVER'], 4), 'ev': evs['OVER/OVER']},
        'UNDER/UNDER': {'prob': round(probs['UNDER/UNDER'], 4), 'ev': evs['UNDER/UNDER']},
        'OVER/UNDER': {'prob': round(probs['OVER/UNDER'], 4), 'ev': evs['OVER/UNDER']},
        'UNDER/OVER': {'prob': round(probs['UNDER/OVER'], 4), 'ev': evs['UNDER/OVER']}
    }

    
def kelly_criterion(probability, payout, stake, kelly_fraction=1.0):
    netProfit = payout - stake
    probabilityOfLoss = 1 - probability
    kelly = (netProfit * probability - probabilityOfLoss) / netProfit
    return max(0, round(kelly * kelly_fraction, 4))
    

In [109]:
# pair EV
# ((player1.prob * player2.prob) * payout) - ((1 - player1.prob * player2.prob) * stake)
pairEV = []
seen = set()

for name1 in current['NAME'].unique():
    for name2 in current['NAME'].unique():
        if name1 == name2:
            continue
        
        name1_sorted, name2_sorted = sorted([name1, name2])
        pair_key = (name1_sorted, name2_sorted)
        if pair_key in seen:
            continue
        seen.add(pair_key)

        
        sim_result = monte_carlo_sim(name1, name2, weight=0.8)
        if sim_result is None:
            continue
        evs = {
            'OVER': sim_result['OVER/OVER']['ev'],
            'UNDER': sim_result['UNDER/UNDER']['ev'],
            'OVER/UNDER': sim_result['OVER/UNDER']['ev'],
            'UNDER/OVER': sim_result['UNDER/OVER']['ev']
        }
        
        probs = {
            'OVER': sim_result['OVER/OVER']['prob'],
            'UNDER': sim_result['UNDER/UNDER']['prob'],
            'OVER/UNDER': sim_result['OVER/UNDER']['prob'],
            'UNDER/OVER': sim_result['UNDER/OVER']['prob']
        }
        
        bestType = max(evs, key=evs.get)
        bestEV = evs[bestType]
        bestProb = probs[bestType]
        
        pairEV.append({
            'PLAYER 1': name1_sorted,
            'PLAYER 1 LINE': pointsProps.loc[pointsProps['NAME'] == name1_sorted, 'LINE'].values[0],
            'PLAYER 2': name2_sorted,
            'PLAYER 2 LINE': pointsProps.loc[pointsProps['NAME'] == name2_sorted, 'LINE'].values[0],
            'TYPE': bestType,
            'EV': round(bestEV, 2),
            'PROBABILITY': round(bestProb, 4), # Probability of both players scoring over their lines
            'KELLY CRITERION': kelly_criterion(bestProb, 300, 100)
        })
todaysPairEV = pd.DataFrame(pairEV).sort_values(by='EV', ascending=False).reset_index(drop=True)
todaysPairEV
# todaysPairEV.to_csv('PROPS_EV/todaysPairEV(6_9_25).csv')

        

,PLAYER 1,PLAYER 1 LINE,PLAYER 2,PLAYER 2 LINE,TYPE,EV,PROBABILITY,KELLY CRITERION
0,Isaiah Hartenstein,7.0,Shai Gilgeous-Alexander,34.5,UNDER/OVER,55.52,0.5184,0.5160
1,Isaiah Hartenstein,7.0,Isaiah Joe,3.5,OVER,41.30,0.4710,0.4684
2,Isaiah Joe,3.5,Shai Gilgeous-Alexander,34.5,UNDER/OVER,36.41,0.4547,0.4520
3,Aaron Nesmith,12.0,Isaiah Hartenstein,7.0,OVER,33.44,0.4448,0.4420
4,Isaiah Hartenstein,7.0,T.J. McConnell,7.0,OVER,33.35,0.4445,0.4417
...,...,...,...,...,...,...,...,...
166,Aaron Wiggins,6.5,Alex Caruso,9.0,UNDER,-16.24,0.2792,0.2756
167,Alex Caruso,9.0,Chet Holmgren,15.0,OVER/UNDER,-16.29,0.2790,0.2754
168,Luguentz Dort,8.5,Obi Toppin,9.5,UNDER,-16.64,0.2779,0.2743
169,Alex Caruso,9.0,Luguentz Dort,8.5,UNDER,-18.62,0.2713,0.2677
